In [4]:
import requests
import pandas as pd
import time
import random
import concurrent.futures


def fetch_stores():
    print("Fetching stores.....")
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-us',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Origin': 'https://www.costco.com',
        'Referer': 'https://www.costco.com/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'client_id': '45823696-9189-482d-89c3-0c067e477ea1',
        'latitude': '28.6305',
        'longitude': '-82.4497',
        'limit': '50',
        'openingDate': '2025-05-02',
    }

    response = requests.get('https://ecom-api.costco.com/warehouseLocatorMobile/v1/warehouses.json', params=params, headers=headers)

    response.raise_for_status()  # Raise an error for bad responses

    data = response.json()

    stores_data = data.get('warehouses', [])
    stores_dict_list = []
    
    for store in stores_data:
        store_dict = {
            'store_id': store.get('warehouseId'),
            'address': store.get('address', None).get('line1', None),
            'city': store.get('address', None).get('city', None),
            'state': store.get('address', None).get('territory', None),
            'zip_code': store.get('address', None).get('postalCode', None),
            'phone_number': store.get('phoneNumber'),
            'latitude': store.get('address', None).get('latitude', None),
            'longitude': store.get('address', None).get('longitude', None),
        }
        stores_dict_list.append(store_dict) 
    stores_df = pd.DataFrame(stores_dict_list)
    stores_df.drop_duplicates(subset='store_id', inplace=True)
    print("Finished fetching stores.")
    return stores_df

df_stores = fetch_stores()

if df_stores is not None:
    df_stores.to_csv('costco_stores.csv', index=False)
    print("Stores data saved to costco_stores.csv")
  

Fetching stores.....
Finished fetching stores.
Stores data saved to costco_stores.csv


In [5]:
#Fetch Categories

def fetch_categories():
    print("Fetching categories...")
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Origin': 'https://www.costco.com',
        'Referer': 'https://www.costco.com/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'x-api-key': '273db6be-f015-4de7-b0d6-dd4746ccd5c3',
    }

    params = {
        'locale': 'en-US',
        'loc': '1250-3pl,1321-wm,1456-3pl,283-wm,561-wm,725-wm,731-wm,758-wm,759-wm,847_0-cor,847_0-cwt,847_0-edi,847_0-ehs,847_0-membership,847_0-mpt,847_0-spc,847_0-wm,847_1-cwt,847_1-edi,847_d-fis,847_lg_n1f-edi,847_NA-cor,847_NA-pharmacy,847_NA-wm,847_ss_u362-edi,847_wp_r458-edi,951-wm,952-wm,9847-wcs,115-bd,1249-wh',
        'chdmegamenu': 'true',
        'userLocation': 'WA',
        'userPostal': '98101',
    }

    response = requests.get(
        'https://search.costco.com/api/apps/www_costco_com/query/www_costco_com_megamenu',
        params=params,
        headers=headers,
    )

    response.raise_for_status()
    data = response.json()
    categories_data = data.get('megaMenu', {})
    categories_dict_list = []
    for category in categories_data:
        subcategories = category.get('children', [])
        if subcategories:
            for subcategory in subcategories:
                subcategory_dict = {
                    'category_id': subcategory.get('url').split('/')[-1],
                    'category_name': subcategory.get('name'),
                    'total_products': int(subcategory.get('count') or 0)
                }
                categories_dict_list.append(subcategory_dict)

    categories_df = pd.DataFrame(categories_dict_list)
    categories_df.drop_duplicates(subset='category_id', inplace=True)
    print("Finished fetching categories.")
    return categories_df

df_categories = fetch_categories()

if df_categories is not None:
    df_categories.to_csv('costco_categories.csv', index=False)
    print("Categories data saved to categories.csv")



Fetching categories...
Finished fetching categories.
Categories data saved to categories.csv


In [6]:
# Fetch Products
LIMIT = 100 

def fetch_products_in_category(row, store_id = '1262'):
    cat_id = row['category_id']
    total_products = row['total_products']
    cat_name = row['category_name']
    

    all_products = []
    skip = 0
    headers = {
    'Accept': 'application/json',
    'Accept-Language': 'es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    'DNT': '1',
    'Origin': 'https://www.costco.com',
    'Referer': 'https://www.costco.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'x-api-key': '273db6be-f015-4de7-b0d6-dd4746ccd5c3',
}


    while True:
        url = (
            'https://search.costco.com/api/apps/www_costco_com/query/www_costco_com_navigation'
            f'?expoption=def&q=*%3A*&locale=en-US&start={skip}&rows={LIMIT}&expand=false&userLocation=WA'
            f'&loc=115-bd%2C1-wh%2C1250-3pl%2C1321-wm%2C1456-3pl%2C283-wm%2C561-wm%2C725-wm%2C731-wm%2C758-wm%2C759-wm%2C847_0-cor'
            '%2C847_0-cwt%2C847_0-edi%2C847_0-ehs%2C847_0-membership%2C847_0-mpt%2C847_0-spc%2C847_0-wm%2C847_1-cwt%2C847_1-edi'
            '%2C847_d-fis%2C847_lg_n1f-edi%2C847_NA-cor%2C847_NA-pharmacy%2C847_NA-wm%2C847_ss_u362-edi%2C847_wp_r458-edi%2C951-wm'
            '%2C952-wm%2C9847-wcs&whloc=1-wh'
            f'&url=/{cat_id}'
            '&fq=%7B!tag%3Ditem_program_eligibility%7Ditem_program_eligibility%3A(%22ShipIt%22)'
            '&chdcategory=true&chdheader=true'
        )
        params = {
            'currenPage': skip // LIMIT, 
        }
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            products = data.get('response', {}).get('docs', [])
            all_products.extend(products)
            print(f'✅ Obtenidos {len(products)} productos de la categoría {cat_name} con skip {skip} en {store_id}')
        except Exception as e:
            print(f'❌ Error al obtener productos de la categoría {cat_name} con skip {skip} en {store_id}: {e}')
            break

        
        if len(products) < LIMIT:
            print(f'🛑 Ya no hay más productos para la categoría {cat_name} en {store_id}')
            sleep_time = random.randint(10, 30)
            print(f'⏳ Esperando {sleep_time}s antes del próximo request...')
            time.sleep(sleep_time)
            break

        skip += LIMIT

        sleep_time = random.randint(10, 30)
        print(f'⏳ Esperando {sleep_time}s antes del próximo request...')
        time.sleep(sleep_time)

    return all_products



def fetch_all_item_info(df_categories, max_workers=5):
    print('Fetching items info...')
    item_info_list = []
    # Crea el grupo de hilos
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #Crea las tareas par cada categoria
        futures = [executor.submit(fetch_products_in_category, row, '1262') for _, row in df_categories.iterrows()]
        
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            try:
                result = future.result()
                item_info_list.extend(result)
                print(f'Completed {i+1}/{len(futures)}')
            except Exception as e:
                print(f"Error in thread: {e}")

    print('Finished fetching item info.')
    return item_info_list



def parse_products_data(responses_list):
    print('Starting products-parsing...')
    item_dict_list = []
    for product in responses_list:
        name = product.get('name')
        id = product.get('id')
        upc_list = product.get('item_manufacturing_skus') or []
        category_path = product.get('categoryPath_ss') or []

        upc = upc_list[0] if upc_list else None
        category_id = category_path[0] if category_path else None
        brand = product.get('Brand_attr')
        weight = product.get('Weight_attr')
        height = product.get('Height_attr')
        width = product.get('Width_attr')
        length = product.get('Length_attr')
        color = product.get('Color_attr')

        item_dict = {
            'id': id,
            'name': name,
            'upc': upc,
            'category': category_id,
            'brand': brand,
            'weight': weight,
            'height': height,
            'width': width,
            'length': length,
            'color': color,
        }

        item_dict_list.append(item_dict)

    print(f'Parsed {len(item_dict_list)} items.')
    return item_dict_list



item_info_list = fetch_all_item_info( df_categories, 6)
item_dict_list = parse_products_data(item_info_list)
df_items = pd.DataFrame(item_dict_list)
df_items.to_csv('costco_productos.csv', index=False)

Fetching items info...
✅ Obtenidos 21 productos de la categoría Kitchen Appliance Packages con skip 0 en 1262
🛑 Ya no hay más productos para la categoría Kitchen Appliance Packages en 1262
⏳ Esperando 16s antes del próximo request...
✅ Obtenidos 25 productos de la categoría Freezers & Ice Makers con skip 0 en 1262
🛑 Ya no hay más productos para la categoría Freezers & Ice Makers en 1262
⏳ Esperando 26s antes del próximo request...
✅ Obtenidos 10 productos de la categoría Commercial & Restaurant Appliances con skip 0 en 1262
🛑 Ya no hay más productos para la categoría Commercial & Restaurant Appliances en 1262
⏳ Esperando 13s antes del próximo request...
✅ Obtenidos 46 productos de la categoría Dishwashers con skip 0 en 1262
🛑 Ya no hay más productos para la categoría Dishwashers en 1262
⏳ Esperando 22s antes del próximo request...
✅ Obtenidos 57 productos de la categoría Heating, Cooling & Air Quality con skip 0 en 1262
🛑 Ya no hay más productos para la categoría Heating, Cooling & Air

In [7]:
def fetch_prices_for_stores(df_categories, store_ids, max_workers=5):
    prices_list = []
    tasks = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        for store_id in store_ids:
            print(f'Fetching prices for store {store_id}...')
            for _, row in df_categories.iterrows():
                tasks.append(executor.submit(fetch_products_in_category, row, store_id))

        for i, future in enumerate(concurrent.futures.as_completed(tasks)):
            try:
                result = future.result()
                prices_list.extend(result)
                print(f'Completed {i+1}/{len(tasks)}')
            except Exception as e:
                print(f"Error in thread: {e}")

    print('Finished fetching prices for all stores.')
    return prices_list



def parse_prices_data(responses_list):
    print('Starting prices-parsing...')
    prices_dict_list = []
    for product in responses_list:
        product_id = product.get('id', None)
        list_price = product.get('item_location_pricing_listPrice', None)
        unit_price = product.get('item_location_pricing_pricePerUnit_price', None)
        sale_price = product.get('item_location_pricing_salePrice', None)
        satus = product.get('item_location_stockStatus', None)
        store_id = product.get('item_location_locationNumber', None)
        item_dict = {
            'product_id': product_id,
            'status': satus,
            'list_price': list_price,
            'unit_price': unit_price,
            'sale_price': sale_price,
            'store_id': store_id
        }
        
        prices_dict_list.append(item_dict)
        

    print(f'Parsed {len(item_dict_list)} items.')
    return prices_dict_list


stores_ids = ['1249', '1235', '1332']
prices_info_list = fetch_prices_for_stores(df_categories, stores_ids, 6)
prices_dict_list = parse_prices_data(prices_info_list)
df_prices = pd.DataFrame( prices_dict_list )
df_prices.to_csv('costco_precios.csv', index=False)

Fetching prices for store 1249...
Fetching prices for store 1235...
Fetching prices for store 1332...
✅ Obtenidos 10 productos de la categoría Commercial & Restaurant Appliances con skip 0 en 1249
🛑 Ya no hay más productos para la categoría Commercial & Restaurant Appliances en 1249
⏳ Esperando 19s antes del próximo request...
✅ Obtenidos 25 productos de la categoría Freezers & Ice Makers con skip 0 en 1249
🛑 Ya no hay más productos para la categoría Freezers & Ice Makers en 1249
⏳ Esperando 19s antes del próximo request...
✅ Obtenidos 21 productos de la categoría Kitchen Appliance Packages con skip 0 en 1249
🛑 Ya no hay más productos para la categoría Kitchen Appliance Packages en 1249
⏳ Esperando 24s antes del próximo request...
✅ Obtenidos 46 productos de la categoría Dishwashers con skip 0 en 1249
🛑 Ya no hay más productos para la categoría Dishwashers en 1249
⏳ Esperando 16s antes del próximo request...
✅ Obtenidos 57 productos de la categoría Heating, Cooling & Air Quality con sk